# Multiple Linear Regression
### Construir el modelo óptimo de RLM utilizando la Eliminación hacia atrás de manera automática
- Como no se tiene reflejado en el dataset el `coeficiente independiente b0` entonces se procede a añadir una `columna llena de 1` (esto debido a que la librería `statsmodels` ya asume que una columna de `1` está asociada al coeficiente independiente) para que el análisis sea adecuado ya que este valor también debe formar parte de cuando se ejecute cualquier técnica de selección de datos, en este caso el de `Eliminación hacia atrás`

- Importante tomar en cuenta que, se debe eliminar una de las columnas generadas por Dummy Variables para evitar la multicolinealidad, por lo cual se pasa el parámetro `drop_first=True` con lo cual, `X` queda solo con 2 columnas nuevas en lugar de 3

### Usando solo p-valores

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

def backwardElimination(x, sl):    
    numVars = len(x[0])    
    for i in range(0, numVars):        
        regressor_OLS = sm.OLS(y, x.tolist()).fit()        
        maxVar = max(regressor_OLS.pvalues).astype(float)        
        if maxVar > sl:            
            for j in range(0, numVars - i):                
                if (regressor_OLS.pvalues[j].astype(float) == maxVar):                    
                    x = np.delete(x, j, 1)    
    regressor_OLS.summary()    
    return x 

dataset = pd.read_csv('50_Startups.csv')
X = pd.get_dummies(dataset.iloc[:, :-1], dtype="int", drop_first=True).values
y = dataset.iloc[:, -1].values

SL = 0.05

X = np.append(arr = np.ones((50,1)).astype(int), values = X, axis = 1)
X_opt = X[:, [0, 1, 2, 3, 4, 5]]
X_Modeled = backwardElimination(X_opt, SL)
print(f"muestra X_modeled:\n{pd.DataFrame(X_Modeled[0:5,:])}\n\n")

X_train, X_test, y_train, y_test = train_test_split(X_Modeled, y, test_size = 0.2, random_state = 0)

regressor = LinearRegression()
regressor.fit(X_train, y_train)

y_pred = regressor.predict(X_test)

pd.set_option('display.precision', 2)
# ndarray.reshape(filas, col)
print(pd.DataFrame(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1), columns=["prediction", "test"]))

muestra X_modeled:
     0          1
0  1.0  165349.20
1  1.0  162597.70
2  1.0  153441.51
3  1.0  144372.41
4  1.0  142107.34


   prediction       test
0   104667.28  103282.38
1   134150.83  144259.40
2   135207.80  146121.95
3    72170.54   77798.83
4   179090.59  191050.39
5   109824.77  105008.31
6    65644.28   81229.06
7   100481.43   97483.56
8   111431.75  110352.25
9   169438.15  166187.94


### Usando p-valores y el valor de  R Cuadrado Ajustado
- De esta manera se tiene un criterio menos rígido a la hora de eliminar variables, por ejemplo si se tiene un p_valor de 0.06, a pesar de que cumple con que es superior a LS la diferencia es muy poca por lo que no necesariamente se debería eliminar esa variable

In [2]:
def backwardElimination(x, SL):    
    numVars = len(x[0])    
    temp = np.zeros((50,6)).astype(int)    
    for i in range(0, numVars):        
        regressor_OLS = sm.OLS(y, x.tolist()).fit()        
        maxVar = max(regressor_OLS.pvalues).astype(float)        
        adjR_before = regressor_OLS.rsquared_adj.astype(float)        
        if maxVar > SL:            
            for j in range(0, numVars - i):                
                if (regressor_OLS.pvalues[j].astype(float) == maxVar):                    
                    temp[:,j] = x[:, j]                    
                    x = np.delete(x, j, 1)                    
                    tmp_regressor = sm.OLS(y, x.tolist()).fit()                    
                    adjR_after = tmp_regressor.rsquared_adj.astype(float)                    
                    if (adjR_before >= adjR_after):                        
                        x_rollback = np.hstack((x, temp[:,[0,j]]))                        
                        x_rollback = np.delete(x_rollback, j, 1)     
                        # print (regressor_OLS.summary())                        
                        return x_rollback                    
                    else:                        
                        continue    
    # regressor_OLS.summary()    
    return x 
 
dataset = pd.read_csv('50_Startups.csv')
X = pd.get_dummies(dataset.iloc[:, :-1], dtype="int", drop_first=True).values
y = dataset.iloc[:, -1].values

SL = 0.05

X = np.append(arr = np.ones((50,1)).astype(int), values = X, axis = 1)
X_opt = X[:, [0, 1, 2, 3, 4, 5]]
X_Modeled = backwardElimination(X_opt, SL)
print(f"muestra X_modeled:\n{pd.DataFrame(X_Modeled[0:5,:])}\n\n")

X_train, X_test, y_train, y_test = train_test_split(X_Modeled, y, test_size = 0.2, random_state = 0)

regressor = LinearRegression()
regressor.fit(X_train, y_train)

y_pred = regressor.predict(X_test)

pd.set_option('display.precision', 2)
# ndarray.reshape(filas, col)
print(pd.DataFrame(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1), columns=["prediction", "test"]))

muestra X_modeled:
     0          1         2
0  1.0  165349.20  471784.0
1  1.0  162597.70  443898.0
2  1.0  153441.51  407934.0
3  1.0  144372.41  383199.0
4  1.0  142107.34  366168.0


   prediction       test
0   102284.66  103282.38
1   133873.92  144259.40
2   134182.14  146121.95
3    73701.09   77798.83
4   180642.25  191050.39
5   114717.23  105008.31
6    68335.08   81229.06
7    97433.47   97483.56
8   114580.92  110352.25
9   170343.32  166187.94
